In [6]:
import pandas as pd
import numpy as np
import sys
import os
import json
import datasets
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import krippendorff

from utils import *

aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']

human_data = datasets.load_dataset('boda/review_evaluation_human_annotation', 'combined_main_aspects', split = 'full').to_pandas()



TYPE = "all"

# labels = pd.re{ad_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv')
# human_data = {}
# for aspect in aspects:
#     human_data[aspect] = []
#     for conf in ['gold','silver', 'hard']:
#         raw_data = datasets.load_dataset('boda/review_evaluation_human_annotation', aspect, split=conf).to_pandas()
#         human_data[aspect].append(raw_data)

#     human_data[aspect] = pd.concat(human_data[aspect])

#     print(len(human_data))


In [7]:
human_data.columns

Index(['review_point', 'paper_id', 'venue', 'focused_review', 'batch',
       'actionability', 'actionability_label', 'actionability_label_type',
       'id', 'grounding_specificity', 'grounding_specificity_label',
       'grounding_specificity_label_type', 'verifiability',
       'verifiability_label', 'verifiability_label_type', 'helpfulness',
       'helpfulness_label', 'helpfulness_label_type'],
      dtype='object')

In [8]:
def process_stat_dict(stat_dict):
    processed_stat_dict = {}
    for k, v in stat_dict.items():
        if 'accuracy' in k:
            continue
        if 'spearman' in k or 'pearson' in k or 'tau' in k:

            v = v[0] if isinstance(v, tuple) else v
        if isinstance(v, float):
            v = round(v, 3)
        processed_stat_dict[k] = v

    return processed_stat_dict

In [14]:
final_data = {
}


for aspect in aspects:
    not_valid_points = 0
    final_data[aspect] = {
          'boda' : [],
    '6686ebe474531e4a1975636f' : [],
    '6740484e188a64793529ee77' : [],
    }

    ### only take the non hard data
    # aspect_data = human_data[human_data[f'{aspect}_label_type'] != 'hard']
    aspect_data = human_data
    
    for i, row in aspect_data.iterrows():
        # data = row[aspect].replace("'", "\"")
        data = row[aspect]
        # data = json.loads(data)
        labels = data['labels']
        annotators = data['annotators']
        for label, annotator in zip(labels, annotators):
            found = False
            if len(annotators) != 3:
                not_valid_points += 1
                continue
            ## search for the annotator in the unique id map

            final_data[aspect][annotator].append(label)

               
    print(f'not valid points for {aspect} is {not_valid_points}')
        


not valid points for actionability is 0
not valid points for grounding_specificity is 0
not valid points for verifiability is 0
not valid points for helpfulness is 0


In [15]:
prinaple_name = {
    'boda' : 'boda',
    '6686ebe474531e4a1975636f' : 'Annotator_X',
    '6740484e188a64793529ee77' : 'Annotator_Y',
}

with open(f'outputs/human_annotators_agreement_{TYPE}.txt', 'w') as f:

    for aspect in aspects:
        
        ## calculate the agreement between each pair of annotators
        annotators = ['boda', '6686ebe474531e4a1975636f', '6740484e188a64793529ee77']
        print(f'Aspect: {aspect}')
        print(f'Number of data points: {len(final_data[aspect][annotators[0]])}')

        f.write(f'Aspect: {aspect}\n')
        f.write(f'Number of data points: {len(final_data[aspect][annotators[0]])}\n')
        f.write(f'===================== PAIRWISE AGREEMENT =====================\n')

        aspect_average_results = {}
        for i in range(len(annotators)):
            for j in range(i+1, len(annotators)):
                annotator1 = annotators[i]
                annotator2 = annotators[j]

                if annotator1 == annotator2:
                    continue
                data1 = final_data[aspect][annotator1]
                data2 = final_data[aspect][annotator2]

                annotator1 = prinaple_name[annotator1]
                annotator2 = prinaple_name[annotator2]

                agreement = process_stat_dict(get_stats(data1, data2, aspect))

                for key in agreement:

                    if key not in aspect_average_results.keys():
                        aspect_average_results[key] = 0.0
                    aspect_average_results[key] += agreement[key]

                f.write(f'Annotator 1: {prinaple_name[annotators[i]]}\n')
                f.write(f'Annotator 2: {prinaple_name[annotators[j]]}\n')
                f.write(json.dumps(agreement, indent=4))
                f.write('\n\n')
            
        for key in aspect_average_results:
            aspect_average_results[key] /= 3

        aspect_average_results =  process_stat_dict(aspect_average_results)
        
        ## calcuate krippendorff's alpha

        annotators_labels = [
            [int(label) if label != "X" else 0 for label in final_data[aspect][annotator]]
            for annotator in annotators
        ]
        alpha  = krippendorff.alpha(annotators_labels,level_of_measurement='ordinal')

        f.write(f'===================== AVERAGE AGREEMENT =====================\n')
        f.write(json.dumps(aspect_average_results, indent=4))
        f.write(f'Krippendorff\'s alpha: {alpha}')
        f.write('\n\n')
        f.write('============================================================\n\n')


        




Aspect: actionability
Number of data points: 1430
Aspect: grounding_specificity
Number of data points: 1430


Aspect: verifiability
Number of data points: 1430
Aspect: helpfulness
Number of data points: 1430
